In [23]:
# %%bash
# apt-get update
# apt-get install g++ openjdk-8-jdk python-dev python3-dev
# pip3 install JPype1
# pip3 install konlpy


In [24]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"


env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [25]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /tmp/mecab-python-0.996
  Created wheel for mecab-python: filename=mecab_python-0.996_ko_0.9.2-cp37-cp37m-linux_x86_64.whl size=141813 sha256=1496f90230330bc7fb5d584e5497ec940ba9c52650ac7b8a7fb6859b48ac34b3
  Stored in directory: /root/.cache/pip/wheels/40/7b/9f/2922869bef86c3354ae7034f7a3647c573ee1997c2dad0290a
Failed to build mecab-python
  Attempting uninstall: mecab-python
    Found existing installation: mecab-python 0.996-ko-0.9.2
    Uninstalling mecab-python-0.996-ko-0.9.2:
      Successfully uninstalled mecab-python-0.996-ko-0.9.2
    Running setup.py install for mecab-python: started
    Running setup.py install for mecab-python: finished with status 'done'


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  DEPRECATION: mecab-python was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [26]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt, Mecab

In [12]:
kkma = Kkma()
mecab = Mecab()
okt = Okt()
text = '네이버 영화 리뷰 감정 분석입니다.'

In [13]:
okt.morphs(text)

['네이버', '영화', '리뷰', '감정', '분석', '입니다', '.']

In [14]:
kkma.morphs(text)

['네이버', '영화', '리뷰', '감정', '분석', '이', 'ㅂ니다', '.']

In [15]:
mecab.morphs(text)

['네이버', '영화', '리뷰', '감정', '분석', '입니다', '.']

In [22]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 5.65 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [41]:
import pandas as pd
import re
train_df = pd.read_csv('./nsmc/ratings_train.txt', sep='\t')
test_df = pd.read_csv('./nsmc/ratings_test.txt', sep='\t')
train_df = train_df.fillna(' ')
test_df = test_df.fillna(' ')

In [43]:
train_df['document'] = train_df['document'].apply(lambda x: re.sub(r'\d+', ' ', x))
test_df['document'] = test_df['document'].apply(lambda x: re.sub(r'\d+', ' ', x))

In [45]:
train_df.drop(['id'], axis=1, inplace=True)
test_df.drop(['id'], axis=1, inplace=True)

In [51]:
from konlpy.tag import Twitter, Okt

In [67]:
# twitter = Twitter()
okt = Okt()
def tw_tokenizer(text):
  # tokens_ko = twitter.morphs(text)
  tokens_ko = okt.morphs(text)
  return tokens_ko

tw_tokenizer(train_df.iloc[0, 0])

['아', '더빙', '..', '진짜', '짜증나네요', '목소리']

In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import pickle

f = open('./review_model', 'rb')
model = pickle.load(f)
f.close()

tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1, 2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])

tfidf_matrix_train = tfidf_vect.transform(train_df['document'])
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

In [22]:
lr_clf = LogisticRegression(random_state=0, solver='liblinear')

params = {
    # 'C':[1, 3.5, 4.5, 5.5, 10]
    'C':[3.5]
}
grid_cv = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfidf_matrix_train, train_df['label'])
print(grid_cv.best_params_, round(grid_cv.best_score_, 4))

In [ ]:
from sklearn.metrics import accuracy_score

best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print(accuracy_score(test_df['label'], preds))

In [77]:
# data = tfidf_vect.transform(['영화가 정말 재미없어요'])
# grid_cv.best_estimator_.predict(data)
model.predict(['영화가 정말 재미있어요'])

array([1])

In [78]:
from sklearn.pipeline import Pipeline

In [79]:
# pipeline = Pipeline(('tfidf_vect', TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1, 2), min_df=3, max_df=0.9)), 
#                     ('lr_clf', LogisticRegression(solver='liblinear')))
params = {
    # 'C':[1, 3.5, 4.5, 5.5, 10]
    'lr_clf__C':[3.5]
}


grid_cv_pipe = GridSearchCV(model, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(train_df['document'], train_df['label'])
print(grid_cv_pipe.best_params_, round(grid_cv_pipe.best_score_, 4))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


{'lr_clf__C': 3.5} 0.8574


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [80]:
pred_pipe = grid_cv_pipe.best_estimator_.predict(test_df['label'])
print(accuracy_score(test_df['label'], pred_pipe))

AttributeError: ignored